# NNS Proposal Visualization

## Fetch and Clean Data

In [1]:
import pandas as pd
import numpy as np
import requests
import warnings
import json
import pickle
import stylecloud
import matplotlib.pyplot as plotly
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import regex
import nltk
import string

from wordcloud import WordCloud
from nltk.corpus import stopwords
from wordcloud import WordCloud, ImageColorGenerator
from pprint import pprint
from tqdm import tqdm
from multiprocessing.pool import ThreadPool
from operator import itemgetter
from PIL import Image
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.util import ngrams

warnings.filterwarnings('ignore')
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/crinstaniev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
base_url = 'https://ic-api.internetcomputer.org'


In [3]:
# get total proposal counts
res = requests.get(f'{base_url}/api/nns/proposals-count')
proposals_count = json.loads(res.text)['proposals_count']

print(f'Total Proposals: {proposals_count}')


Total Proposals: 87283


In [4]:
# fetch proposals
def get_proposal(id):
    url = base_url + '/api/v3/proposals/' + str(id)
    res = requests.get(url)
    res_dict = json.loads(res.text)
    return res_dict


def get_neuron(id):
    url = f'https://ic-api.internetcomputer.org/api/v3/neurons/{id}'
    res = requests.get(url)
    res_dict = json.loads(res.text)
    return res_dict


In [5]:
# proposals = []
# proposal_ids = [i for i in range(proposals_count)]


# def dummy(id):
#     res = get_proposal(id)
#     proposals.append(dict(
#         id=id,
#         data=res
#     ))
#     print(f'fetched: proposal {id}')

# # enable multithreading for faster download
# pool = ThreadPool(50)

# pool.imap_unordered(dummy, proposal_ids)


In [6]:
# pickle.dump(proposals, open('nns_data_raw.pkl', 'wb'))


In [7]:
data = pickle.load(open('nns_data_raw.pkl', 'rb'))
data.sort(key=itemgetter('id'))
json.dump(data, open('proposals.json', 'w'))


In [8]:
# filter empty data
proposals_no_empty = []

for item in data:
    if (item['data'].get('code') != 404):
        proposals_no_empty.append(
            item['data']
        )
json.dump(proposals_no_empty, open('proposals_no_empty.json', 'w'))


In [9]:
df = pd.read_json('proposals_no_empty.json')


In [10]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69989 entries, 0 to 69988
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   action                      69989 non-null  object        
 1   action_nns_function         69142 non-null  object        
 2   deadline_timestamp_seconds  51182 non-null  float64       
 3   decided_timestamp_seconds   69989 non-null  int64         
 4   executed_timestamp_seconds  69983 non-null  float64       
 5   failed_timestamp_seconds    66391 non-null  float64       
 6   id                          69989 non-null  int64         
 7   known_neurons_ballots       69989 non-null  object        
 8   latest_tally                69989 non-null  object        
 9   payload                     69986 non-null  object        
 10  proposal_id                 69989 non-null  int64         
 11  proposal_timestamp_seconds  69989 non-null  int64     

## Wordcloud - Proposal Summary

In [11]:
with open('proposal_summary_text.txt', 'w') as f:
    f.write(' '.join(list(df['summary'].values)))


In [12]:
stylecloud.gen_stylecloud(file_path='proposal_summary_text.txt',
                          icon_name='fas fa-folder',
                          palette='colorbrewer.diverging.Spectral_11',
                          background_color='black',
                          gradient='horizontal',
                          output_name='proposal_summary_wordcloud.png',
                          size=1024)


In [13]:
# cleaning data for wordcloud
df_wordcloud = df[['topic', 'summary']]
df_wordcloud = df_wordcloud.groupby('topic').aggregate(' '.join).reset_index()
df_wordcloud.to_csv('topic_summary_joined.csv')

# remove non-utf8 characters


def remove_non_utf8(x: str):
    x = x.encode('utf-8', errors='ignore').decode('utf-8')
    x = regex.sub(r'[^\x00-\x7f]', u'', x)
    x = regex.sub(r"http\S+", "", x)
    x = x.translate(str.maketrans('', '', string.punctuation))
    x = word_tokenize(x)
    x = [word for word in x if not word in stopwords.words('english')]

    return (" ").join(x)


df_wordcloud['summary'] = df_wordcloud['summary'].apply(remove_non_utf8)
df_wordcloud.to_csv('topic_summary_joined.csv')
df_wordcloud


topic  \
0                TOPIC_EXCHANGE_RATE   
1                   TOPIC_GOVERNANCE   
2                          TOPIC_KYC   
3  TOPIC_NETWORK_CANISTER_MANAGEMENT   
4            TOPIC_NETWORK_ECONOMICS   
5                   TOPIC_NODE_ADMIN   
6        TOPIC_NODE_PROVIDER_REWARDS   
7       TOPIC_PARTICIPANT_MANAGEMENT   
8            TOPIC_SUBNET_MANAGEMENT   

                                             summary  
0  The ICPXDR conversion rate set 404904 The ICPX...  
1  SetAuthorizedSubnetworks SetAuthorizedSubnetwo...  
2  The proposal unlocks neurons associated listed...  
3  Upgrade ledger canister git commit 8a560f9510b...  
4  ClearProvisionalWhitelist Initialize rewards t...  
5  Remove nodes registry Remove nodes registry Re...  
6  Proposal mint 1 ICP node provider reward sixix...  
7  Add node provider Blockchange Ventures II prin...  
8  Add verifiedapplication subnet number 5 commit...

In [14]:
def generate_wordcloud(df_wordcloud, topics):

    words = []
    for topic in topics:
        words.append(
            df_wordcloud.query(f"topic == '{topic}'")['summary'].values[0]
        )

    all_words = (' ').join(words)

    with open('all_words.txt', 'w') as f:
        f.write(all_words)

    wc = WordCloud(
        background_color='black',
        repeat=True,
        width=1024 * 2,
        height=768 * 2,
        max_words=200,
        colormap='Set2',
        font_path='impact.ttf'
    )
    wc.generate(all_words)

    return wc


In [15]:
# wc = generate_wordcloud(
#     df_wordcloud=df_wordcloud,
#     topics=[
#         'TOPIC_EXCHANGE_RATE',
#         'TOPIC_GOVERNANCE',
#         # 'TOPIC_KYC',
#         # 'TOPIC_NETWORK_CANISTER_MANAGEMENT',
#         # 'TOPIC_NETWORK_ECONOMICS',
#         # 'TOPIC_NODE_ADMIN',
#         # 'TOPIC_NODE_PROVIDER_REWARDS',
#         # 'TOPIC_PARTICIPANT_MANAGEMENT',
#         'TOPIC_SUBNET_MANAGEMENT',
#     ]
# )
# plt.figure(dpi=300)
# plt.axis('off')
# plt.imshow(wc)
# plt.show()


In [20]:
# trying out pyecharts
import pyecharts.options as opts
from pyecharts.charts import WordCloud
from pyecharts.globals import ThemeType


def generate_word_freq(df_wordcloud, topics):

    words = []
    for topic in topics:
        words.append(
            df_wordcloud.query(f"topic == '{topic}'")['summary'].values[0]
        )

    all_words = [ele for ele in words if len(ele) > 20]
    all_words = (' ').join(all_words).upper()
    all_words = regex.sub(r'\b[0-9]+\b\W*', '', all_words)
    all_words = regex.sub(r'(?:^| )\w(?:$| )', ' ', all_words)

    def replace_digit(string):
        string = regex.sub(r'\d', '', string).strip()
        return string

    all_words = all_words.upper()

    tokens = nltk.word_tokenize(all_words)
    text = nltk.Text(tokens)
    freq = nltk.Counter(text).items()
    res = []
    for item in freq:
        res.append(item)

    res.sort(key=lambda x: x[1], reverse=True)
    return res

_topics = [
        'TOPIC_EXCHANGE_RATE',
        'TOPIC_GOVERNANCE',
        'TOPIC_KYC',
        'TOPIC_NETWORK_CANISTER_MANAGEMENT',
        'TOPIC_NETWORK_ECONOMICS',
        'TOPIC_NODE_ADMIN',
        'TOPIC_NODE_PROVIDER_REWARDS',
        'TOPIC_PARTICIPANT_MANAGEMENT',
        'TOPIC_SUBNET_MANAGEMENT',
    ]

# data = generate_word_freq(
#     df_wordcloud=df_wordcloud,
#     topics=[
#         # 'TOPIC_EXCHANGE_RATE',
#         'TOPIC_GOVERNANCE',
#         # 'TOPIC_KYC',
#         # 'TOPIC_NETWORK_CANISTER_MANAGEMENT',
#         # 'TOPIC_NETWORK_ECONOMICS',
#         # 'TOPIC_NODE_ADMIN',
#         # 'TOPIC_NODE_PROVIDER_REWARDS',
#         # 'TOPIC_PARTICIPANT_MANAGEMENT',
#         # 'TOPIC_SUBNET_MANAGEMENT',
#     ]
# )

# (
#     WordCloud(init_opts=opts.InitOpts())
#     .add(
#         series_name="Keywords",
#         data_pair=data,
#         word_size_range=[20, 80],
#         textstyle_opts=opts.TextStyleOpts(
#             font_family="impact",
#             font_weight='bold',
#             color='red'
#         ),
#         rotate_step=90
#     )
#     .set_global_opts(
#         # title_opts=opts.TitleOpts("Theme-dark"),
#         title_opts=opts.TitleOpts(
#             title="Keywords", title_textstyle_opts=opts.TextStyleOpts(font_size=23),
#         ),
#         tooltip_opts=opts.TooltipOpts(is_show=True),
#     )
#     .render('wordclouds/TOPIC_EXCHANGE_RATE.html')
# )

for topic in _topics:
    data = generate_word_freq(
        df_wordcloud=df_wordcloud,
        topics=[
            topic
        ]
    )

    (
        WordCloud(init_opts=opts.InitOpts())
        .add(
            series_name="Keywords",
            data_pair=data,
            word_size_range=[20, 80],
            textstyle_opts=opts.TextStyleOpts(
                font_family="impact",
                font_weight='bold',
                color='red'
            ),
            rotate_step=90
        )
        .set_global_opts(
            # title_opts=opts.TitleOpts("Theme-dark"),
            title_opts=opts.TitleOpts(
                title="Keywords", title_textstyle_opts=opts.TextStyleOpts(font_size=23),
            ),
            tooltip_opts=opts.TooltipOpts(is_show=True),
        )
        .render(f'wordclouds/{topic}.html')
    )



## Area Chart - Topics River

In [17]:
topic_time_df = df[['proposal_id', 'topic',
                    'updated_at']].set_index('proposal_id')

topic_time_df['updated_at'] = topic_time_df['updated_at'].dt.floor('d')

topic_time_stats = topic_time_df.groupby(
    'updated_at').value_counts().reset_index()
topic_time_stats.columns = ['date', 'topic', 'count']
topic_time_stats


date                              topic  count
0    2021-08-05                TOPIC_EXCHANGE_RATE    265
1    2021-08-05            TOPIC_SUBNET_MANAGEMENT    100
2    2021-08-05        TOPIC_NODE_PROVIDER_REWARDS    100
3    2021-08-05                   TOPIC_GOVERNANCE     86
4    2021-08-05  TOPIC_NETWORK_CANISTER_MANAGEMENT     59
...         ...                                ...    ...
1135 2022-10-05                TOPIC_EXCHANGE_RATE    143
1136 2022-10-06                TOPIC_EXCHANGE_RATE    198
1137 2022-10-06            TOPIC_SUBNET_MANAGEMENT     47
1138 2022-10-06  TOPIC_NETWORK_CANISTER_MANAGEMENT      3
1139 2022-10-06                   TOPIC_NODE_ADMIN      1

[1140 rows x 3 columns]

In [18]:
fig = px.area(topic_time_stats, x='date', y='count',
              color='topic', template='plotly_dark')
fig.update_yaxes(type='log', range=[0, 4])

with open('timeriver.html', 'w') as f:
    f.write(fig.to_html())

fig.show()


In [19]:
px.data.gapminder()

country continent  year  lifeExp       pop   gdpPercap iso_alpha  \
0     Afghanistan      Asia  1952   28.801   8425333  779.445314       AFG   
1     Afghanistan      Asia  1957   30.332   9240934  820.853030       AFG   
2     Afghanistan      Asia  1962   31.997  10267083  853.100710       AFG   
3     Afghanistan      Asia  1967   34.020  11537966  836.197138       AFG   
4     Afghanistan      Asia  1972   36.088  13079460  739.981106       AFG   
...           ...       ...   ...      ...       ...         ...       ...   
1699     Zimbabwe    Africa  1987   62.351   9216418  706.157306       ZWE   
1700     Zimbabwe    Africa  1992   60.377  10704340  693.420786       ZWE   
1701     Zimbabwe    Africa  1997   46.809  11404948  792.449960       ZWE   
1702     Zimbabwe    Africa  2002   39.989  11926563  672.038623       ZWE   
1703     Zimbabwe    Africa  2007   43.487  12311143  469.709298       ZWE   

      iso_num  
0           4  
1           4  
2           4  
3           4  
4           4  
...       ...  
1699      716  
1700      716  
1701      716  
1702      716  
1703      716  

[1704 rows x 8 columns]

In [20]:
topic_time_df = df[['proposal_id', 'topic',
                    'updated_at']].set_index('proposal_id')

topic_time_df['updated_at'] = topic_time_df['updated_at'].dt.floor('d')

topic_time_stats = topic_time_df.groupby(
    'updated_at').value_counts().unstack(fill_value=0).reset_index()
topics = list(topic_time_stats.columns[1:])

topic_time_stats['sum'] = 0

for topic in topics:
    topic_time_stats['sum'] += topic_time_stats[topic]

topic_time_stats


topic updated_at  TOPIC_EXCHANGE_RATE  TOPIC_GOVERNANCE  TOPIC_KYC  \
0     2021-08-05                  265                86          7   
1     2021-08-06                  144                 0          0   
2     2021-08-07                  144                 0          0   
3     2021-08-08                  144                 0          0   
4     2021-08-09                  144                 0          0   
..           ...                  ...               ...        ...   
423   2022-10-02                  144                 1          0   
424   2022-10-03                  144                 0          0   
425   2022-10-04                  143                 1          0   
426   2022-10-05                  143                 0          0   
427   2022-10-06                  198                 0          0   

topic  TOPIC_NETWORK_CANISTER_MANAGEMENT  TOPIC_NETWORK_ECONOMICS  \
0                                     59                        1   
1                                      0                        0   
2                                      0                        0   
3                                      0                        0   
4                                      0                        0   
..                                   ...                      ...   
423                                    0                        0   
424                                    0                        0   
425                                    2                        0   
426                                    0                        0   
427                                    3                        0   

topic  TOPIC_NODE_ADMIN  TOPIC_NODE_PROVIDER_REWARDS  \
0                     9                          100   
1                     0                            0   
2                     0                            0   
3                     0                            0   
4                     0                            0   
..                  ...                          ...   
423                   0                            0   
424                   0                            0   
425                   0                            0   
426                   0                            0   
427                   1                            0   

topic  TOPIC_PARTICIPANT_MANAGEMENT  TOPIC_SUBNET_MANAGEMENT  sum  
0                                15                      100  642  
1                                 0                        8  152  
2                                 0                        0  144  
3                                 0                        0  144  
4                                 0                        1  145  
..                              ...                      ...  ...  
423                               0                       21  166  
424                               0                       14  158  
425                               0                        1  147  
426                               0                        0  143  
427                               0                       47  249  

[428 rows x 11 columns]

In [21]:
from plotly.subplots import make_subplots

# fig = go.Figure()
fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

for topic in topics:
    fig.add_trace(go.Scatter(
        x=topic_time_stats['updated_at'],
        y=topic_time_stats[topic],
        mode='lines',
        line=dict(width=0.5),
        stackgroup='one',
        groupnorm='percent',
        name=topic,
    ), row=2, col=1)

fig.add_traces(
    go.Line(x=topic_time_stats['updated_at'], y=topic_time_stats['sum'], name='Sum of all Topics'))

fig.update_layout(dict(template='plotly_dark',
                  title='Proposal Topic Changes Over Time', width=1200, height=800))

# fig.update_layout(yaxis_range=(0, 100))
fig.update_xaxes(title='Time')

fig['layout']['yaxis2'].update(title='Percentage', range=[60, 100])
fig['layout']['yaxis1'].update(title='Count')

fig.show()
with open('topic_area_chart.html', 'w') as f:
    f.write(fig.to_html())


/Users/crinstaniev/opt/anaconda3/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




## Chord & Terminus

In [22]:
proposer_network = df[['proposal_id', 'proposer',
                       'known_neurons_ballots', 'updated_at']]

# drop proposals with no votes
proposer_network = proposer_network[proposer_network['known_neurons_ballots'].apply(
    len) != 0]
proposer_network['proposer'] = proposer_network['proposer'].astype(int)

# build proposer roster
voters = proposer_network['known_neurons_ballots'].values.flatten()
voters_temp = set()
voters_id = set()
for voter_group in tqdm(voters):
    for voter in voter_group:
        voters_id.add(int(voter.get('id')))
        voter = (voter.get('id'), voter.get('name'))
        voters_temp.add(voter)

voters = voters_temp

proposer_set = set()
for proposal in proposer_network.itertuples():
    proposer_id = proposal.proposer
    proposer_set.add(int(proposer_id))


100%|██████████| 69959/69959 [00:00<00:00, 238241.02it/s]


In [23]:
chord_df = pd.DataFrame(0, columns=voters_id, index=proposer_set)

for proposal in tqdm(proposer_network.itertuples(), total=len(proposer_network)):
    voters = proposal.known_neurons_ballots
    proposer_id = int(proposal.proposer)
    for voter in voters:
        voter_id = int(voter.get('id'))
        vote = voter.get('vote')
        chord_df.loc[proposer_id, voter_id] += 1

100%|██████████| 69959/69959 [00:14<00:00, 4739.74it/s] 


In [24]:
links = []
nodes = []

for item in chord_df.itertuples():
    proposer = f'Proposer {item.Index}'
    nodes.append(dict(
        name=proposer,
        group='left'
    ))
    for voter in chord_df.columns:
        amount = int(chord_df.loc[item.Index, voter])
        voter = f'Voter {voter}'
        links.append(dict(
                source=voter,
                target=proposer,
                value=amount
        ))

for voter in chord_df.columns:
    voter = f'Voter {voter}'
    nodes.append(dict(
        name=voter,
        group='right'
    ))


In [25]:
from plotapi import SplitChord, Terminus, Chord
import plotapi

plotapi.api_key("d494c31b-ce51-4470-aa8c-7749ac52ac0b")

Your PlotAPI API key has been saved in your local environment. You will not need to set it again.


In [26]:
SplitChord(
    links,
    nodes,
    directed=True,
    conjunction='to',
    verb='give',
    noun='votes',
    color='purple_red'
).to_html('chord.html')


In [27]:
few_links = list(filter(lambda x: x['value'] > 1000, links))


In [28]:
Terminus(
    few_links,
    show_stats=False
).to_html('terminus.html')

In [29]:
len(few_links)

67

### Network

In [36]:
df[['summary', 'topic']]

summary  \
0      Upgrade ledger canister to git commit 8a560f95...   
1      Upgrade registry canister to git commit 8a560f...   
2      Upgrade governance canister to git commit 8a56...   
3      Upgrade root canister to git commit 8a560f9510...   
4      Upgrade cycles-minting canister to git commit ...   
...                                                  ...   
69984                                                      
69985                                                      
69986                                                      
69987  Halt subnet fuqsr for resharing of ECDSA key t...   
69988                                                      

                                   topic  
0      TOPIC_NETWORK_CANISTER_MANAGEMENT  
1      TOPIC_NETWORK_CANISTER_MANAGEMENT  
2      TOPIC_NETWORK_CANISTER_MANAGEMENT  
3      TOPIC_NETWORK_CANISTER_MANAGEMENT  
4      TOPIC_NETWORK_CANISTER_MANAGEMENT  
...                                  ...  
69984                TOPIC_EXCHANGE_RATE  
69985                TOPIC_EXCHANGE_RATE  
69986                TOPIC_EXCHANGE_RATE  
69987            TOPIC_SUBNET_MANAGEMENT  
69988                TOPIC_EXCHANGE_RATE  

[69989 rows x 2 columns]

In [45]:
df_network = df[df['topic'] != 'TOPIC_EXCHANGE_RATE']['summary']

In [174]:
network_all_words = ('. '.join(list(df_network.values)))
network_all_words = regex.sub(r'\b[0-9]+\b\W*', '', network_all_words)
network_all_words = regex.sub(r'(?:^| )\w(?:$| )', ' ', network_all_words)

In [175]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections

import nltk
from nltk import bigrams
from nltk.corpus import stopwords
import re
import networkx as nx
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

import warnings
warnings.filterwarnings("ignore")

nltk.download('wordnet')
sns.set(font_scale=1.5)
sns.set_style("whitegrid")

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/crinstaniev/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [176]:
def remove_url(txt):
    """Replace URLs found in a text string with nothing 
    (i.e. it will remove the URL from the string).

    Parameters
    ----------
    txt : string
        A text string that you want to parse and remove urls.

    Returns
    -------
    The same txt string with url's removed.
    """
    
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    no_url = url_pattern.sub(r'', txt)

    return no_url

In [177]:
def stem_tokens(tokens):
    stemmer = nltk.stem.PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    return tokens

def lemmatize_tokens(tokens):
    lemmatizer = nltk.stem.WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens

In [269]:
text_no_urls = remove_url(network_all_words)
text_no_urls = text_no_urls.encode('utf-8', errors='ignore').decode('utf-8')
words_in_text = text_no_urls.split()
stop_words = set(stopwords.words('english'))
words_nsw = [word for word in words_in_text if not word in stop_words]
words_nsw = [ele for ele in words_nsw if len(ele) < 20]


In [270]:
lemmatizer = WordNetLemmatizer()
words_nsw_nc = [lemmatizer.lemmatize(w) for w in words_nsw]
words_nsw_nc = [
    word for word in words_nsw_nc if word not in string.punctuation]


In [271]:
bigram = list(nltk.bigrams(words_nsw_nc))
bigram_counts = collections.Counter(bigram).most_common(100)
# bigram_df = pd.DataFrame(bigram_counts.most_common(50), columns=['bigram', 'count'])

In [272]:
import plotly.graph_objects as go

import networkx as nx
import igviz as ig

In [273]:
G = nx.Graph()
for edge in bigram_counts:
    link = edge[0]
    count = edge[1]
    
    G.add_edge(link[0], link[1], weight=count)

In [278]:
fig = ig.plot(
    G,
    node_opacity=0.8,
)
fig.update_layout(
    dict(
        width=800,
        height=600,
    )
)

with open('network.html', 'w') as f:
    f.write(fig.to_html())

fig.show()


plotly.graph_objs._figure.Figure